In [2]:
#load and install required packages

In [3]:
#install required packages
%pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite
%pip install scikit_learn
%pip install nltk
%pip install seaborn

  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to c:\users\ewout\appdata\local\temp\pip-install-p7yf31zl\sklearn-crfsuite_19a6982a97a1496db94d9250e5bb5157
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git 'C:\Users\ewout\AppData\Local\Temp\pip-install-p7yf31zl\sklearn-crfsuite_19a6982a97a1496db94d9250e5bb5157'

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ewout\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# import required libraries
import pandas as pd 
import nltk
import sklearn
import scipy.stats
import numpy as np
import seaborn as sns
import sklearn_crfsuite

#from matplotlib 
from matplotlib import pyplot as plt

#from itertools
from itertools import chain

#from sklearn
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [5]:
#load the data and ontologies

In [6]:
#this function turns the file into a list. 
def file2list(fileLocation):
    outputList = []
    with open(fileLocation, 'r', encoding='utf8') as myfile:
        sentences = myfile.read().split('\n\n')
        for sentence in sentences:
                sentenceList = []
                words = sentence.split('\n')
                for word in words:
                    wordsList = []
                    attributes = word.split(' ')
                    for attribute in attributes:
                        wordsList.append(attribute)
                    sentenceList.append(wordsList)
                outputList.append(sentenceList)
    
    return outputList

In [8]:

# train =  file2list('D:\\phd-data\\NER-annotation-data\\Dutch\\5-folds-with-pos-with-cut-sentences\\fold1.txt') 
# test =  file2list('D:\\phd-data\\NER-annotation-data\\Dutch\\5-folds-with-pos-with-cut-sentences\\fold2.txt')

#load the datasets, with training documents as train, and test documents and test
fold = "fold1"
train =  file2list('data/5-folds-train-test-split/'+ fold +'/train.txt') 
test =  file2list('data/5-folds-train-test-split/'+ fold +'/test.txt')

#remove empty line at the end of the file as this breaks the code
test.pop() 
train.pop()

[['=', 'RB', 'O'],
 ['Unidentified', 'NNP', 'O'],
 ['Quantification', 'NNP', 'O'],
 ['presented', 'VBD', 'O'],
 ['as', 'IN', 'O'],
 ['no./wt', 'NN', 'O'],
 ['(', '(', 'O'],
 ['weights', 'NNS', 'O'],
 ['in', 'IN', 'O'],
 ['grams', 'NN', 'O'],
 [')', ')', 'O'],
 [',', ',', 'O'],
 ['with', 'IN', 'O'],
 ['the', 'DT', 'O'],
 ['exception', 'NN', 'O'],
 ['of', 'IN', 'O'],
 ['amber/glass', 'NN', 'O'],
 ['beads', 'NNS', 'O'],
 ['and', 'CC', 'O'],
 ['metalwork', 'NN', 'O'],
 ['which', 'WDT', 'O'],
 ['is', 'VBZ', 'O'],
 ['presented', 'VBN', 'O'],
 ['by', 'IN', 'O'],
 ['number', 'NN', 'O'],
 ['only', 'RB', 'O'],
 ['Note', 'NNP', 'O'],
 ['-', ':', 'O'],
 ['only', 'RB', 'O'],
 ['those', 'DT', 'O'],
 ['ASGs', 'NNP', 'O'],
 ['that', 'WDT', 'O'],
 ['have', 'VBP', 'O'],
 ['produced', 'VBN', 'O'],
 ['finds', 'NNS', 'O'],
 ['are', 'VBP', 'O'],
 ['listed', 'VBN', 'O'],
 ['below', 'IN', 'O'],
 ['.', '.', 'O']]

In [9]:
%%time 
# calculates the time to open the file
        
#train the NER on the list. there is one set of test and one of training. often 20:80 split
train_sent = train
test_sent = test   # tests the sent (input) of the given list as defined above
train_sent[0] # displayes the first 10 rows in the bio. - each row hs the token (effectively word), followed by pos?, and the bio label


CPU times: total: 0 ns
Wall time: 0 ns


[['The', 'DT', 'O'],
 ['modern', 'JJ', 'B-PER'],
 ['ploughsoil', 'NN', 'O'],
 ['was', 'VBD', 'O'],
 ['300', 'CD', 'O'],
 ['mm', 'JJ', 'O'],
 ['deep', 'NN', 'O'],
 ['and', 'CC', 'O'],
 ['directly', 'RB', 'O'],
 ['overlay', 'VB', 'O'],
 ['archaeological', 'JJ', 'O'],
 ['deposits', 'NNS', 'O'],
 ['.', '.', 'O']]

In [11]:
#time to load the ontologies 
"""
#artifact ontology
artifacts = pd.read_csv(r'ontologies\artifacts.csv') #open the file
artifacts[artifacts.columns[2]]= artifacts[artifacts.columns[2]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
artifacts_list = artifacts[artifacts.columns[2]].values.tolist() #turns the items into a list
print(artifacts_list) # outputs the list of terms to see what sort of data it contains
print("\n""\n")
"""
artifacts_list = []
with open(r"ontologies\german_mobile_objects.txt", "r", encoding="utf-8") as f:
    for i in f:
        artifacts_list.append(i.strip('\n'))
print(artifacts_list[0:100])
print("\n\n")
"""

#Taxon ontology
# taxon = pd.read_csv(r'ontologies\Taxon.tsv', sep="\t", error_bad_lines=False) #open the file
# taxon[taxon.columns[8]]= taxon[taxon.columns[8]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
# taxon_list = taxon[taxon.columns[8]].values.tolist() #turns the items into a list
# print(taxon_list[0:100]) # outputs the list of terms to see what sort of data it contains
# print("\n""\n")

#Animal ontology
animal = pd.read_csv(r'ontologies\VernacularName.tsv', sep="\t") #open the file
animal[animal.columns[2]]= animal[animal.columns[2]].str.lower() #takes the second column (the one with the entities) and makes it all lower case
animal_list = animal[animal.columns[2]].values.tolist() #turns the items into a list
print(animal_list[0:100]) # outputs the list of terms to see what sort of data it contains
print("\n""\n")
"""

#Periods ontology - for notes see above
#periods = pd.read_csv(r'ontologies\Periods.csv')
#periods[periods.columns[1]]= periods[periods.columns[1]].str.lower()
#periods_list = periods[periods.columns[1]].values.tolist()
periods_list = []
with open(r"ontologies\periodo-dataset.txt", "r", encoding="utf-8") as f:
    for i in f:
        periods_list.append(i.strip('\n'))
print(periods_list[0:100])
print("\n\n")

#Context ontology - for notes see artifact 
"""
context = pd.read_csv(r'ontologies\context.csv')
context[context.columns[0]]= context[context.columns[0]].str.lower().str.replace('*','').str.replace('<','').str.replace('>','') #replace all other symbols
context[context.columns[0]]= context[context.columns[0]].replace(to_replace =':.*',value='',regex=True) # replace the words after a colon
context_list = context[context.columns[0]].values.tolist()
print(context_list[0:100])
print("\n""\n")
"""

FileNotFoundError: [Errno 2] No such file or directory: 'ontologies\\german_mobile_objects.txt'

In [113]:
# turn each word into a list of features
def word2features(sent, i): 
    word = sent[i][0] #this is the token
    postag = sent[i][1] #this is the Part of Speech tag
    
     #this tells if the token is in the ontology or not 
    if word in artifacts_list: 
        in_artifacts = True 
    else:
        in_artifacts = False
    
#     if word in taxon_list: 
#         in_taxon = True 
#     else:
#         in_taxon = False
    
#    if word in animal_list: 
#        in_animal = True 
#    else:
#        in_animal = False
  
    if word in periods_list: 
        in_periods = True 
    else:
        in_periods = False

#    if word in context_list: 
#        in_context = True 
#    else:
#        in_context = False
    
    #time to give each token some information     
    features = { # these are all default. 
        'bias': 1.0, # bias is just 1. 
        'word.lower()': word.lower(), # tells if the token is lower case 
        'word[-3:]': word[-3:], # takes the last four letters - the suffix
#        'Word.in_artifacts': in_artifacts, #is the token in the artifact ontology
#         'Word.in_taxon': in_taxon, #is the token in the artifact ontology
#         'Word.in_animal': in_animal, #is the token in the periods ontology 
#        'Word.in_periods': in_periods, #is the token in the periods ontology
#        'Word.in_context': in_context, #is the token in the evidence ontology
        'word.isupper()': word.isupper(), # tells if the whole token is uppercase 
        'word.istitle()': word.istitle(), # tells if the token is capital first letter
        'postag': postag,  # what is its label - Part-Of-Speech Tagger
        'postag[:2]': postag[:2],  #Takes the first three letters of the tag
        'word.isdigit()': word.isdigit(),
    }
    if i > 0: #if the token is not at the start of a sentence
        word1 = sent[i-1][0] # works out details of the token before - this is to understand the context 
        postag1 = sent[i-1][1] #what is the postag of the word before
        wordbefore = (sent[i-1][0]+ ' ' +sent[i][0]).lower #this is the token and the token before
        #if this word and word before is in the ontology then 
        if wordbefore in artifacts_list: 
            wordbefore_in_artifacts = True 
        else:
            wordbefore_in_artifacts = False

#         if wordbefore in taxon_list: 
#             wordbefore_in_taxon = True 
#         else:
#             wordbefore_in_taxon = False

#        if wordbefore in animal_list: 
#            wordbefore_in_animal = True 
#            print(word)
#        else:
#            wordbefore_in_animal = False

        if wordbefore in periods_list: 
            wordbefore_in_periods = True 
        else:
            wordbefore_in_periods = False

#        if wordbefore in context_list: 
#            wordbefore_in_context = True 
#        else:
#            wordbefore_in_context = False
        features.update({
            '-1:word.lower()': word1.lower(), # tells if the token is lower case
            '-1:word.istitle()': word1.istitle(), # tells if the token is capital first letter
            '-1:word.isdigit()': word1.isdigit(), # tells if the toekn is only numbers
            '-1:word.isupper()': word1.isupper(),# tells if the whole token is uppercase
#            'wordbefore_in_artifacts': wordbefore_in_artifacts,
#             'wordbefore_in_taxon': wordbefore_in_taxon,
#             'wordbefore_in_animal': wordbefore_in_animal,
#            'wordbefore_in_periods': wordbefore_in_periods,
#            'wordbefore_in_context': wordbefore_in_context,
            '-1:postag': postag1, # what was its POS tag
            '-1:postag[:2]': postag1[:2], #what is the first three POS tag of the word before
        })
    
        if i > 1: #if the token is not at the start of a sentence
                word3 = sent[i-1][0] # works out details of the token before - this is to understand the context 
                postag3 = sent[i-1][1] #what is the postag of the word before
                two_words_before = (sent[i-1][0]+ ' ' +sent[i-1][0]+ ' ' +sent[i][0]).lower #this is the token and the token before
                #if this word and word before is in the ontology then 
                if two_words_before in artifacts_list: 
                    two_words_before_in_artifacts = True 
                else:
                    two_words_before_in_artifacts = False

#                 if two_words_before in taxon_list: 
#                     two_words_before_in_taxon = True 
#                 else:
#                     two_words_before_in_taxon = False

#                if two_words_before in animal_list: 
#                    two_words_before_in_animal = True 
#                    print(word)
#                else:
#                    two_words_before_in_animal = False

                if two_words_before in periods_list: 
                    two_words_before_in_periods = True 
                else:
                    two_words_before_in_periods = False

#                if two_words_before in context_list: 
#                    two_words_before_in_context = True 
#                else:
#                    two_words_before_in_context = False
                features.update({
                    '-1:word.lower()': word1.lower(), # tells if the token is lower case
                    '-1:word.istitle()': word1.istitle(), # tells if the token is capital first letter
                    '-1:word.isdigit()': word1.isdigit(), # tells if the toekn is only numbers
                    '-1:word.isupper()': word1.isupper(),# tells if the whole token is uppercase
#                    'two_words_before_in_artifacts': two_words_before_in_artifacts,
#                     'two_words_before_in_taxon': two_words_before_in_taxon,
#                     'two_words_before_in_animal': two_words_before_in_animal,
#                    'two_words_before_in_periods': two_words_before_in_periods,
#                    'two_words_before_in_context': two_words_before_in_context,
                    '-2-postag': postag3, # what was its POS tag
                    '-2:postag[:2]': postag3[:2], #what is the first three POS tag of the word before
                })
        else:
            features['BOS2'] = True # if word is the beggining of sentence label it as so         
    else:
        features['BOS'] = True # if word is the beggining of sentence label it as so 
        
    if i < len(sent)-1: # is the word at the end of the sentence. sme as above after
        wordafter= (sent[i][0]+ ' ' +sent[i+1][0]).lower
        #this tells if the token AFTER and each token combined is in the ontology or not
        if wordafter in artifacts_list: 
            wordafter_in_artifacts = True 
        else:
            wordafter_in_artifacts = False

#         if wordafter in taxon_list: 
#             wordafter_in_taxon = True 
#         else:
#             wordafter_in_taxon = False

#        if wordafter in animal_list: 
#            wordafter_in_animal = True 
#        else:
#            wordafter_in_animal = False

        if wordafter in periods_list: 
            wordafter_in_periods = True 
        else:
            wordafter_in_periods = False

#        if wordafter in context_list: 
#            wordafter_in_context = True 
#        else:
#            wordafter_in_context = False
        word2 = sent[i+1][0]  
        postag2 = sent[i+1][1] 
        features.update({
#            'wordafter_in_artifacts': wordafter_in_artifacts,
#             'wordafter_in_taxon': wordafter_in_taxon,
#             'wordafter_in_animal': wordafter_in_animal,
#            'wordafter_in_periods': wordafter_in_periods,
#            'wordafter_in_context': wordafter_in_context,
            '+2:word.lower()': word2.lower(),
            '+2:word.istitle()': word2.istitle(),
            '+2:word.isupper()': word2.isupper(),
            '+2:postag': postag2,
            '+2:postag[:2]': postag2[:2],
        })
        
        if i < len(sent)-2: #if the token is not at the start of a sentence
                word4 = sent[i-1][0] # works out details of the token before - this is to understand the context 
                postag4 = sent[i-1][1] #what is the postag of the word before
                two_words_after = (sent[i][0]+ ' ' +sent[i+1][0]+ ' ' +sent[i+1][0]).lower#this is the token and the token before
                #if this word and word before is in the ontology then 
                if two_words_after in artifacts_list: 
                    two_words_after_in_artifacts = True 
                else:
                    two_words_after_in_artifacts = False

#                 if two_words_after in taxon_list: 
#                     two_words_after_in_taxon = True 
#                 else:
#                     two_words_after_in_taxon = False

#                if two_words_after in animal_list: 
#                    two_words_after_in_animal = True 
#                    print(word)
#                else:
#                    two_words_after_in_animal = False

                if two_words_after in periods_list: 
                    two_words_after_in_periods = True 
                else:
                   two_words_after_in_periods = False

#                if two_words_after in context_list: 
#                    two_words_after_in_context = True 
#                else:
#                    two_words_after_in_context = False
                features.update({
                    '-1:word.lower()': word4.lower(), # tells if the token is lower case
                    '-1:word.istitle()': word4.istitle(), # tells if the token is capital first letter
                    '-1:word.isdigit()': word4.isdigit(), # tells if the toekn is only numbers
                    '-1:word.isupper()': word4.isupper(),# tells if the whole token is uppercase
#                    'two_words_after_in_artifacts': two_words_after_in_artifacts,
#                     'two_words_after_in_taxon': two_words_after_in_taxon,
#                     'two_words_after_in_animal': two_words_after_in_animal,
#                    'two_words_after_in_periods': two_words_after_in_periods,
#                    'two_words_after_in_context': two_words_after_in_context,
                    '-2-postag': postag4, # what was its POS tag
                    '-2:postag[:2]': postag4[:2], #what is the first three POS tag of the word before
                })
        else:
            features['EOS2'] = True # if word is the beggining of sentence label it as so       
        
        
        
    else:
        features['EOS'] = True # if word is the end of sentence label it as so         
   

    if i < len(sent)-1 and i > 0: # is the word at the end of the sentence. sme as above after
            word3 = sent[i+1][0]  
            postag3 = sent[i+1][1] 
            wordsorround = (sent[i-1][0]+ ' ' +sent[i][0]+ ' ' +sent[i+1][0]).lower
            if wordsorround in artifacts_list: 
                wordsorround_in_artifacts = True 
            else:
                wordsorround_in_artifacts = False

#             if wordsorround in taxon_list: 
#                 wordsorround_in_taxon = True 
#             else:
#                 wordsorround_in_taxon = False

#            if wordsorround in animal_list: 
#                wordsorround_in_animal = True 
#            else:
#                wordsorround_in_animal = False

            if wordsorround in periods_list: 
                wordsorround_in_periods = True 
            else:
                wordsorround_in_periods = False

#            if wordsorround in context_list: 
#                wordsorround_in_context = True 
#            else:
#                wordsorround_in_context = False
            features.update({
#                'wordsorround_in_artifacts': wordsorround_in_artifacts,
#                 'wordsorround_in_taxon': wordsorround_in_taxon,
#                 'wordsorround_in_animal': wordsorround_in_animal,
#                'wordsorround_in_periods': wordsorround_in_periods,
#                'wordsorround_in_context': wordsorround_in_context,
            })
            if i < len(sent)-2 and i > 1: # is the word at the end of the sentence. sme as above after
                word5 = sent[i+1][0]  
                postag5 = sent[i+1][1] 
                twowordsorround = (sent[i-2][0]+ ' ' +sent[i-1][0]+ ' ' +sent[i][0]+ ' ' +sent[i+1][0]+ ' ' +sent[i+2][0]).lower
                if twowordsorround in artifacts_list: 
                    twowordsorround_in_artifacts = True 
                else:
                    twowordsorround_in_artifacts = False

#                 if twowordsorround in taxon_list: 
#                     twowordsorround_in_taxon = True 
#                 else:
#                     twowordsorround_in_taxon = False

#                if twowordsorround in animal_list: 
#                    twowordsorround_in_animal = True 
#                else:
#                    twowordsorround_in_animal = False

                if twowordsorround in periods_list: 
                    twowordsorround_in_periods = True 
                else:
                    twowordsorround_in_periods = False

#                if twowordsorround in context_list: 
#                    twowordsorround_in_context = True 
#                else:
#                    twowordsorround_in_context = False
                features.update({
#                    'twowordsorround_in_artifacts': twowordsorround_in_artifacts,
#                     'twowordsorround_in_taxon': twowordsorround_in_taxon,
#                     'twowordsorround_in_animal': twowordsorround_in_animal,
#                    'twowordsorround_in_periods': twowordsorround_in_periods,
#                    'twowordsorround_in_context': twowordsorround_in_context,                   
                })
            else:
                    features['OWS'] = True # if word is the end of sentence label it as so 
    else:
            features['OWS'] = True # if word is the end of sentence label it as so 
            
    return features # output these details
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))] #output for each word

def sent2labels(sent):
    #print(sent)
    for word in sent:
        if len(word) > 3:
            print(word)
    return [label for token, postag, label in sent] #output for each token

def sent2tokens(sent):
    return [token for token, postag, label in sent] #output for ???


In [114]:
#print(sent2features[:2])

In [115]:
#sent2features(train_sent[0:1])[0]
[sent2features(s) for s in train_sent[0:1]]

[[{'bias': 1.0,
   'word.lower()': 'linearbandkeramik',
   'word[-3:]': 'mik',
   'word.isupper()': False,
   'word.istitle()': True,
   'postag': 'NOUN',
   'postag[:2]': 'NO',
   'word.isdigit()': False,
   'BOS': True,
   '+2:word.lower()': 'aus',
   '+2:word.istitle()': False,
   '+2:word.isupper()': False,
   '+2:postag': 'ADP',
   '+2:postag[:2]': 'AD',
   '-1:word.lower()': '.',
   '-1:word.istitle()': False,
   '-1:word.isdigit()': False,
   '-1:word.isupper()': False,
   '-2-postag': 'PUNCT',
   '-2:postag[:2]': 'PU',
   'OWS': True},
  {'bias': 1.0,
   'word.lower()': 'aus',
   'word[-3:]': 'aus',
   'word.isupper()': False,
   'word.istitle()': False,
   'postag': 'ADP',
   'postag[:2]': 'AD',
   'word.isdigit()': False,
   '-1:word.lower()': 'linearbandkeramik',
   '-1:word.istitle()': True,
   '-1:word.isdigit()': False,
   '-1:word.isupper()': False,
   '-1:postag': 'NOUN',
   '-1:postag[:2]': 'NO',
   'BOS2': True,
   '+2:word.lower()': 'meindling',
   '+2:word.istitle()

In [116]:
#time to train the NER

In [117]:
%%time
X_train = [sent2features(s) for s in train_sent] # for the token train the ner on the learned set
y_train = [sent2labels(s) for s in train_sent] # for the POS tag train the ner on the learned set


CPU times: total: 17.1 s
Wall time: 17.3 s


In [118]:
X_test = [sent2features(s) for s in test_sent] # for the token train the ner on the test set
y_test = [sent2labels(s) for s in test_sent] # for the POS tag train the ner on the test set

In [119]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1,  
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
#crf.fit(X_train, y_train)
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

CPU times: total: 36.6 s
Wall time: 37.2 s


In [120]:
labels = list(crf.classes_) # get the list of all labels
labels.remove('O') # remove the ones where bio is o - not got a postag
labels # show what the labels are 

['B-ART',
 'B-LOC',
 'B-CON',
 'B-PER',
 'I-ART',
 'B-MAT',
 'I-PER',
 'I-MAT',
 'I-CON',
 'I-LOC',
 'B-SPE',
 'I-SPE']

In [121]:
#now to evaluate its success

In [122]:
#calculate the f1 score
y_pred = crf.predict(X_test) # work out and predict what is the likely token
metrics.flat_f1_score(y_test, y_pred, # work out what the likely postag will be, and give it a f1 score
                      average='weighted', labels=labels)

0.5159440641573898

In [123]:
#calculate the metrics table
# group B and I results - this isnt needed, but orders the list
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
#print(metrics.classification_report(y_test, y_pred))
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels
))
with open ("classification report "+ fold +".txt", 'w', encoding="utf-8") as f:
    f.write(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels
))

              precision    recall  f1-score   support

       B-ART       0.65      0.22      0.33       965
       I-ART       0.29      0.11      0.16       175
       B-CON       0.80      0.53      0.64       993
       I-CON       0.00      0.00      0.00       170
       B-LOC       0.65      0.30      0.41      1030
       I-LOC       0.45      0.27      0.34       279
       B-MAT       0.57      0.19      0.29       156
       I-MAT       0.00      0.00      0.00         5
       B-PER       0.91      0.64      0.75      1564
       I-PER       0.70      0.50      0.59       535
       B-SPE       0.53      0.41      0.46        22
       I-SPE       0.00      0.00      0.00         1

   micro avg       0.75      0.42      0.54      5895
   macro avg       0.46      0.27      0.33      5895
weighted avg       0.71      0.42      0.52      5895



In [124]:
output = "Word  actual_tag  prediction"
for x in range(len(test_sent)):
    for i in test_sent[x]:
        testWord = i[0]
        testTag = i[2]
        for j in y_pred[x]:
            prediction = j
        output += "\n" + testWord + "  " + testTag + "  " + prediction
        #print(testWord + "," + testTag + "," + prediction)
    output += "\n"
with open ("output "+ fold +".txt", 'w', encoding="utf-8") as f:
    f.write(output)
print(output)

Word  actual_tag  prediction
Einleitung  O  O
Der  O  O
ursprüngliche  O  O
Entstehungskontext  O  O
eines  O  O
archäologischen  O  O
Befunds  O  O
und  O  O
der  O  O
in  O  O
ihm  O  O
fassbaren  O  O
Objekte  O  O
spielt  O  O
eine  O  O
primäre  O  O
Rolle  O  O
für  O  O
die  O  O
Interpretation  O  O
materieller  O  O
Hinterlassenschaften  O  O
.  O  O

Gräber  B-CON  O
stellen  O  O
in  O  O
erster  O  O
Linie  O  O
das  O  O
Resultat  O  O
eines  O  O
sozial  O  O
prekären  O  O
und  O  O
emotionalen  O  O
Ereignisses  O  O
in  O  O
einer  O  O
Gemeinschaft  O  O
dar  O  O
–  O  O
dem  O  O
Tod  O  O
eines  O  O
ihrer  O  O
Mitglieder  O  O
.  O  O

Aus  O  O
archäologischer  O  O
Perspektive  O  O
repräsentieren  O  O
Bestattungen  B-CON  O
so  O  O
eine  O  O
primäre  O  O
Quelle  O  O
für  O  O
die  O  O
Erforschung  O  O
des  O  O
Todesverständnisses  O  O
in  O  O
vergangenen  O  O
Gesellschaften  O  O
.  O  O

In  O  O
der  O  O
deutschsprachigen  O  O
Frühmittelalterfor

In [21]:
#work out which of the transitions are most likely in descending order
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

#what are the 20 most likely transitions
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

#what are the 20 least likely transitions
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-ART  -> I-ART   4.436642
I-PER  -> I-PER   4.391448
B-PER  -> I-PER   4.295925
B-SPE  -> I-SPE   4.243853
B-MAT  -> I-MAT   4.213733
B-CON  -> I-CON   4.093637
I-CON  -> I-CON   4.024182
I-LOC  -> I-LOC   3.953437
I-SPE  -> I-SPE   3.873839
I-MAT  -> I-MAT   3.873479
B-ART  -> I-ART   3.862515
B-LOC  -> I-LOC   3.425596
O      -> O       2.633426
O      -> B-PER   1.000630
O      -> B-LOC   0.577695
O      -> B-ART   0.551631
B-MAT  -> B-ART   0.415516
B-PER  -> O       0.387884
B-CON  -> O       0.355969
B-MAT  -> B-CON   0.300831

Top unlikely transitions:
B-PER  -> I-LOC   -2.269256
B-MAT  -> I-ART   -2.271779
I-PER  -> I-CON   -2.290625
I-PER  -> I-ART   -2.333396
I-ART  -> B-PER   -2.382300
B-CON  -> I-LOC   -2.483171
B-CON  -> I-PER   -2.656976
B-LOC  -> I-PER   -2.694411
B-LOC  -> I-CON   -2.931987
B-ART  -> I-PER   -3.087528
B-LOC  -> I-ART   -3.161349
B-SPE  -> B-PER   -3.363866
O      -> I-MAT   -3.488953
B-PER  -> I-CON   -3.520569
B-PER  -> I-ART  

In [22]:
#What aspects of the terms make it likely to be that tag
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

#what are the 30 most likely aspects
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

#what 30 aspects make it least likely to be that term
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
7.066906 B-MAT    word.lower():hölzernen
6.663383 B-SPE    word.lower():menschlichen
6.579406 B-SPE    word.lower():rinder
6.393056 B-LOC    word.lower():weingarten
6.323662 B-SPE    word.lower():roggen
6.173232 B-LOC    word.lower():hoch-elten
5.960042 B-MAT    word.lower():eisen
5.912307 B-PER    word.lower():römisch
5.860907 B-MAT    word.lower():hölzerner
5.857536 B-CON    word.lower():scherbenpackung
5.797684 B-SPE    word.lower():menschen
5.768394 B-CON    word.lower():grabhügeln
5.693129 B-LOC    word.lower():dortmund-asseln
5.672671 B-SPE    word.lower():rindern
5.662072 B-PER    word[-3:]:kum
5.655167 B-SPE    word.lower():erbsen
5.643437 B-PER    word.lower():endneolithische
5.625611 B-SPE    word.lower():ziegen
5.583702 B-CON    word.lower():brunnen
5.582071 B-PER    word.lower():jahrhundert
5.455079 B-CON    word.lower():brunnenkasten
5.418401 B-LOC    word.lower():schwanfelder
5.395805 B-MAT    word.lower():granaten
5.372987 B-CON    word[-3:]:rab
5.361908 B-

In [23]:
## optional: do hyperparameter tuning

In [24]:
%%time 
# this is to work out the best parameters for the testing. not needed yet but can increase the results 
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer,
                        return_train_score = True)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
# crf = rs.best_estimator_  # shows that the best params are 
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
#rs.cv_results_["mean_test_score"]
rs.cv_results_.keys()

In [ ]:
#_x = [s.parameters['c1'] for s in rs.grid_scores_]
#_y = [s.parameters['c2'] for s in rs.grid_scores_]
#_c = [s.mean_validation_score for s in rs.grid_scores_]

_x = [s['c1'] for s in rs.cv_results_['params']]
_y = [s['c2'] for s in rs.cv_results_['params']]
_c = [s for s in rs.cv_results_['mean_train_score']]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))